In [ ]:
%pip install bastionai

In [2]:
%pip install torch==1.12.0 torchvision

  Using cached torchvision-0.13.1-cp310-cp310-manylinux1_x86_64.whl (19.1 MB)
  Using cached Pillow-9.2.0-cp310-cp310-manylinux_2_28_x86_64.whl (3.2 MB)
  Using cached torchvision-0.13.0-cp310-cp310-manylinux1_x86_64.whl (19.1 MB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.datasets import CIFAR100
from torchvision.transforms import ToTensor, Normalize, Compose
from torch.utils.data import DataLoader

transform = Compose([
    ToTensor(),
    Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

train_dataset = CIFAR100("data", train=True, transform=transform, download=True)
test_dataset = CIFAR100("data", train=False, transform=transform, download=True)

train_dataloader = DataLoader(train_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)

model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
from bastionai.client import Connection

with Connection("localhost", 50051) as client:
    remote_dataloader = client.RemoteDataLoader(train_dataloader, test_dataloader, name="SMSSpamCollection")
    
    remote_learner = client.RemoteLearner(
        model,
        remote_dataloader,
        loss="cross_entropy",
        model_name="DistilBERT",
        progress = False,
    )

    remote_learner.fit(nb_epochs=100, eps=6.0)
    # remote_learner.test(metric="accuracy")
    
    # trained_model = remote_learner.get_model()

KeyboardInterrupt: 

In [6]:
remote_learner.log

[]